# DenseNet-121 Fine-Tuning - Bone Fracture Detection

## Module 3: Deep Learning Fine-Tuning

Complete DenseNet-121 fine-tuning pipeline with two-phase training on Google Colab GPU.

## Features:
- ✅ Free GPU access (T4, 16GB VRAM)
- ✅ Upload dataset from PC
- ✅ Two-phase fine-tuning (Classifier + Full model)
- ✅ Automatic model download
- ✅ Training visualization and metrics


In [ ]:
# Install dependencies for DenseNet fine-tuning
%pip install torch torchvision --index-url https://download.pytorch.org/whl/cu118 -q
%pip install scikit-learn pandas matplotlib seaborn tqdm opencv-python opencv-contrib-python scikit-image scipy -q

print("✅ Dependencies installed!")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 30.4 MB/s eta 0:00:0000:01
✅ Dependencies installed!


In [ ]:
# Check GPU/CPU
import torch
print("="*60)
print("DEVICE INFORMATION")
print("="*60)

if torch.cuda.is_available():
    print(f"✅ CUDA Available: True")
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
    vram_gb = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"VRAM: {vram_gb:.2f} GB")
    print("✅ GPU is ready for fine-tuning!")
    device_type = "GPU"
    batch_size = 16 if vram_gb >= 8 else 8
else:
    print("ℹ️  CUDA Available: False")
    print("✅ CPU mode enabled - Training will work but will be slower")
    print("💡 Tip: For faster training, go to Runtime > Change runtime type > GPU (T4)")
    device_type = "CPU"
    batch_size = 4
print("="*60)


DEVICE INFORMATION
✅ CUDA Available: True
GPU Name: Tesla T4
VRAM: 15.83 GB
✅ GPU is ready for training!
Recommended: YOLOv8s


## Step 2: Upload Dataset from PC

Choose one option to upload your dataset:


In [ ]:
# Option A: Upload Dataset ZIP from PC
from google.colab import files
from pathlib import Path
import zipfile
import os

print("="*80)
print("UPLOAD DATASET FROM PC")
print("="*80)
print("📁 Select your dataset ZIP file...")
print("   (Should contain: train/, valid/, test/ folders with images and labels)")
print("="*80)

uploaded = files.upload()

# Create project directory
project_dir = Path('/content/bone_fracture_detection')
project_dir.mkdir(exist_ok=True)
data_dir = project_dir / 'data' / 'archive'
data_dir.mkdir(parents=True, exist_ok=True)

for filename in uploaded.keys():
    print(f"\n✅ Uploaded: {filename}")
    
    # Extract if ZIP
    if filename.endswith('.zip'):
        print(f"📦 Extracting {filename}...")
        with zipfile.ZipFile(filename, 'r') as zip_ref:
            zip_ref.extractall(data_dir)
        print(f"✅ Extracted to: {data_dir}")
        
        # Find the dataset folder
        extracted_folders = [f for f in data_dir.iterdir() if f.is_dir()]
        if extracted_folders:
            dataset_folder = extracted_folders[0]
            print(f"✅ Dataset folder found: {dataset_folder.name}")
    else:
        print(f"⚠️  Please upload a ZIP file")

# Check if dataset exists
dataset_path = data_dir / "bone fracture detection.v4-v4.yolov8"
if not dataset_path.exists():
    # Try to find any folder with data.yaml
    for folder in data_dir.rglob("data.yaml"):
        dataset_path = folder.parent
        print(f"✅ Found dataset at: {dataset_path}")
        break

if dataset_path.exists() and (dataset_path / "data.yaml").exists():
    print(f"\n✅ Dataset ready at: {dataset_path}")
    print(f"✅ data.yaml found: {(dataset_path / 'data.yaml').exists()}")
else:
    print(f"\n⚠️  Dataset not found. Please upload the dataset ZIP file.")


Select your dataset file (ZIP recommended)...


## Step 3: Upload Training Scripts from PC

Upload the required Python files for fine-tuning:


In [ ]:
# Option B: Upload Training Scripts from PC
from pathlib import Path
import shutil

print("="*80)
print("UPLOAD TRAINING SCRIPTS")
print("="*80)
print("📁 Upload these files from your PC:")
print("   1. run_complete_training.py")
print("   2. src/ folder (with bonefracture module)")
print("="*80)

# Create src directory structure
src_dir = project_dir / 'src' / 'bonefracture'
src_dir.mkdir(parents=True, exist_ok=True)

print("\n📤 Upload run_complete_training.py:")
uploaded_scripts = files.upload()

for filename in uploaded_scripts.keys():
    if filename.endswith('.py'):
        dest = project_dir / filename
        shutil.move(filename, dest)
        print(f"✅ Uploaded: {filename} → {dest}")
    elif filename.endswith('.zip') and 'src' in filename.lower():
        # Extract src folder
        with zipfile.ZipFile(filename, 'r') as zip_ref:
            zip_ref.extractall(project_dir)
        print(f"✅ Extracted src folder from: {filename}")

# Check if required files exist
required_files = [
    project_dir / 'run_complete_training.py',
    src_dir / 'bone_yolo_dataset.py'
]

print("\n📋 Checking required files:")
all_exist = True
for file_path in required_files:
    if file_path.exists():
        print(f"✅ {file_path.name}")
    else:
        print(f"❌ {file_path.name} - MISSING")
        all_exist = False

if all_exist:
    print("\n✅ All required files are ready!")
else:
    print("\n⚠️  Some files are missing. Please upload them.")


Configuration (GPU):
  Model: YOLOv8s
  Epochs: 10
  Batch Size: 16 (adjusted for GPU)
  Image Size: 640


## Step 4: Configure Fine-Tuning Parameters


In [ ]:
# Configure Fine-Tuning Parameters
import sys
from pathlib import Path
import os

# Add project to path
sys.path.insert(0, str(project_dir))

# Find dataset path
dataset_path = None
for possible_path in [
    data_dir / "bone fracture detection.v4-v4.yolov8",
    data_dir / "BoneFractureYolo8"
]:
    if possible_path.exists() and (possible_path / "data.yaml").exists():
        dataset_path = possible_path
        break

# Also search recursively
if not dataset_path:
    for yaml_file in data_dir.rglob("data.yaml"):
        dataset_path = yaml_file.parent
        break

print("="*80)
print("FINE-TUNING CONFIGURATION")
print("="*80)

if dataset_path and dataset_path.exists():
    # Convert to string and normalize path for Linux (Colab)
    dataset_path_str = str(dataset_path).replace('\\', '/')
    print(f"✅ Dataset found: {dataset_path_str}")
    
    # Verify dataset structure
    train_images = Path(dataset_path) / 'train' / 'images'
    if not train_images.exists():
        print(f"⚠️  Warning: train/images not found at {train_images}")
        print("   Checking alternative locations...")
        # Try to find train folder
        for split in ['train', 'Train', 'TRAIN']:
            alt_path = Path(dataset_path) / split / 'images'
            if alt_path.exists():
                print(f"   ✅ Found at: {alt_path}")
                break
    else:
        print(f"✅ Dataset structure verified: {train_images}")
    
    # Update run_complete_training.py with correct paths
    training_script = project_dir / 'run_complete_training.py'
    if training_script.exists():
        with open(training_script, 'r', encoding='utf-8') as f:
            script_content = f.read()
        
        # Update dataset path - replace all possible Windows path formats
        replacements = [
            (r"DATASET_ROOT = r'data\\archive\\bone fracture detection.v4-v4.yolov8'", f"DATASET_ROOT = r'{dataset_path_str}'"),
            (r"DATASET_ROOT = r'data\archive\bone fracture detection.v4-v4.yolov8'", f"DATASET_ROOT = r'{dataset_path_str}'"),
            (r"DATASET_ROOT = r'/content/bone_fracture_detection/data/archive/bone fracture detection.v4-v4.yolov8'", f"DATASET_ROOT = r'{dataset_path_str}'"),
            (r'DATASET_ROOT = r"data\\archive\\bone fracture detection.v4-v4.yolov8"', f'DATASET_ROOT = r"{dataset_path_str}"'),
            (r'DATASET_ROOT = r"data\archive\bone fracture detection.v4-v4.yolov8"', f'DATASET_ROOT = r"{dataset_path_str}"'),
        ]
        
        for old_path, new_path in replacements:
            if old_path in script_content:
                script_content = script_content.replace(old_path, new_path)
                print(f"✅ Replaced path: {old_path[:50]}...")
        
        # Also replace any remaining Windows-style paths in the script
        script_content = script_content.replace('data\\archive', dataset_path_str.replace('/content/bone_fracture_detection/', ''))
        
        # Update batch size
        script_content = script_content.replace(
            "BATCH_SIZE = 16",
            f"BATCH_SIZE = {batch_size}"
        )
        
        # Ensure NUM_WORKERS is set correctly for Colab
        if "NUM_WORKERS = 0 if not torch.cuda.is_available() else 4" in script_content:
            script_content = script_content.replace(
                "NUM_WORKERS = 0 if not torch.cuda.is_available() else 4",
                "NUM_WORKERS = 4 if torch.cuda.is_available() else 0"
            )
        
        with open(training_script, 'w', encoding='utf-8') as f:
            f.write(script_content)
        
        print(f"✅ Updated training script with dataset path: {dataset_path_str}")
        print(f"✅ Batch size set to: {batch_size} (for {device_type})")
    else:
        print("⚠️  run_complete_training.py not found!")
        print("   Please upload it in Step 3")
else:
    print("⚠️  Dataset not found!")
    print("   Please upload dataset in Step 2")
    print(f"   Searched in: {data_dir}")

print("\n📊 Training Configuration:")
print(f"   Device: {device_type}")
print(f"   Batch Size: {batch_size}")
print(f"   Phase 1 Epochs: 10 (Classifier training)")
print(f"   Phase 2 Epochs: 10 (Full fine-tuning)")
print(f"   Total Time: ~1-2 hours on GPU")
print("="*80)


DATASET CONFIGURATION
⚠️  No dataset found!

📋 DATASET UPLOAD OPTIONS:

1️⃣  OPTION A: Use Project Dataset
   - Go to Runtime → Run all cells above
   - Click folder icon (left sidebar)
   - Upload your 'data/archive/' folder from the project
   - This notebook will auto-detect it

2️⃣  OPTION B: Mount Google Drive
   - If your dataset is in Google Drive
   - Run Cell 5 (Google Drive mount)
   - Copy dataset to /content/bone_fracture_detection/data/

3️⃣  OPTION C: Clone from GitHub
   - Uncomment the git clone line in Cell 12
   - Full project will be cloned with dataset

⏭️  Skipping training - dataset not available.
Please upload dataset using one of the options above and re-run this cell.


## Step 5: Run DenseNet-121 Fine-Tuning

This will run two-phase training:
- **Phase 1**: Train classifier with frozen backbone (10 epochs)
- **Phase 2**: Fine-tune all layers with differential LR (10 epochs)


In [ ]:
# Run Fine-Tuning
import os
import sys
from pathlib import Path

os.chdir(str(project_dir))
sys.path.insert(0, str(project_dir))

training_script = project_dir / 'run_complete_training.py'

if training_script.exists():
    print("="*80)
    print("STARTING DENSENET-121 FINE-TUNING")
    print("="*80)
    print("This will take ~1-2 hours for 20 epochs total (10+10)...")
    print("="*80)
    print()
    
    # Verify dataset path before running
    with open(training_script, 'r', encoding='utf-8') as f:
        script_content = f.read()
    
    # Extract DATASET_ROOT from script
    import re
    match = re.search(r"DATASET_ROOT\s*=\s*r?['\"]([^'\"]+)['\"]", script_content)
    if match:
        dataset_root = match.group(1)
        dataset_path_check = Path(dataset_root)
        if not dataset_path_check.exists():
            print(f"⚠️  ERROR: Dataset path does not exist: {dataset_root}")
            print("Please check Step 4 configuration and ensure dataset is uploaded correctly.")
        else:
            print(f"✅ Dataset path verified: {dataset_root}")
            # Execute the training script
            exec(open(training_script, encoding='utf-8').read())
    else:
        print("⚠️  Could not find DATASET_ROOT in training script")
        print("Please check the script configuration")
else:
    print("⚠️  run_complete_training.py not found!")
    print("Please upload it in Step 3")


In [ ]:
## Step 6: View Results & Download Models


In [ ]:
# Display Results and Download Models
from IPython.display import Image, display
from pathlib import Path
from google.colab import files
import json

results_dir = project_dir / 'training_results'
checkpoints_dir = project_dir / 'checkpoints'

print("="*80)
print("TRAINING RESULTS")
print("="*80)

# Display training history plot
history_plot = results_dir / 'complete_training_history.png'
if history_plot.exists():
    display(Image(str(history_plot)))
    print("✅ Training history plot displayed!")
else:
    print("⚠️  Training history plot not found yet (may still be generating)")

# Download best models
models_to_download = [
    checkpoints_dir / 'best_model_phase_1.pth',
    checkpoints_dir / 'best_model_phase_2.pth',
    checkpoints_dir / 'final_model_complete.pth'
]

print("\n📥 Downloading models...")
downloaded_count = 0
for model_path in models_to_download:
    if model_path.exists():
        files.download(str(model_path))
        print(f"✅ Downloaded: {model_path.name}")
        downloaded_count += 1
    else:
        print(f"⚠️  Not found: {model_path.name}")

if downloaded_count == 0:
    print("⚠️  No models found. Training may still be in progress.")

# Download training results JSON
results_json = results_dir / 'complete_training_results.json'
if results_json.exists():
    files.download(str(results_json))
    print("✅ Training results JSON downloaded!")
    
    # Display results summary
    try:
        with open(results_json, 'r') as f:
            results = json.load(f)
        print("\n📊 Final Results Summary:")
        print(f"  Phase 1 Best Val Accuracy: {results.get('phase1_best_val_acc', 'N/A'):.4f}")
        print(f"  Phase 2 Best Val Accuracy: {results.get('phase2_best_val_acc', 'N/A'):.4f}")
        print(f"  Final Test Accuracy: {results.get('final_test_acc', 'N/A'):.4f}")
    except Exception as e:
        print(f"⚠️  Could not parse results: {e}")
else:
    print("⚠️  Training results JSON not found yet (may still be generating)")

print("="*80)
